# Samenvatting

## Association Rules indentificeren
Het identificeren van associatieregels gaat in **drie stappen**:
1. De gegevens in het juiste formaat zetten met datamanagementtechnieken.
2. De **frequenties van de combinaties** berekenen die voorkomen in de transacties.
3. Met de frequenties van deze combinaties kun je daarna **associatieregels** bepalen.

*Association Rules zullen worden beschreven als **AR***

## Stap 1: Datamanagement
Stel:
- Je krijgt van de verkoopafdeling een bestand met verkopen die er als volgt uit kan zien:

| KassaTicket | Product   |
|-------------|-----------|
| 1000123     | Printer   |
| 1000123     | Cartridge |
| 1000123     | Balpen    |
| 1000124     | Papier    |
| 1000124     | Cartridge |
| 1000125     | Papier    |
| 1000125     | Cartridge |
| 1000126     | Printer   |
| 1000126     | Cartridge |
| 1000127     | Printer   |
| 1000127     | Cartridge |
| 1000128     | Balpen    |
| 1000129     | Papier    |
| 1000129     | Cartridge |
| 1000129     | Balpen    |
| 1000130     | Cartridge |
| 1000131     | Printer   |
| 1000131     | Balpen    |
| 1000132     | Papier    |

In [1]:
import pandas as pd

verkopen = pd.DataFrame({
    'KassaTicket': [1000123, 1000123, 1000123, 1000124, 1000124, 1000125, 1000125, 1000126, 1000126, 1000127, 1000127,
                    1000128, 1000129, 1000129, 1000129, 1000130, 1000131, 1000131, 1000132],
    'Product': ['Printer', 'Cartridge', 'Balpen', 'Papier', 'Cartridge', 'Papier', 'Cartridge', 'Printer', 'Cartridge',
                'Printer', 'Cartridge', 'Balpen', 'Papier', 'Cartridge', 'Balpen', 'Cartridge', 'Printer', 'Balpen',
                'Papier']
})
verkopen.set_index('KassaTicket', inplace=True)

verkopen

,Product
KassaTicket,
1000123,Printer
1000123,Cartridge
1000123,Balpen
1000124,Papier
1000124,Cartridge
1000125,Papier
1000125,Cartridge
1000126,Printer
1000126,Cartridge


De aankoop van één product komt in dit bestand overeen met één rij in het bestand: als iemand gelijktijdig meerdere dingen koopt krijg je meerdere lijnen.
Het algoritme dat we gaan gebruiken heeft een ander dataformaat nodig: deze heeft één rij voor alle aankopen van één klant nodig.
⇒

In [2]:
transacties = pd.get_dummies(verkopen, columns=['Product'], prefix='', prefix_sep='').groupby(level='KassaTicket').sum()

transacties

,Balpen,Cartridge,Papier,Printer
KassaTicket,,,,
1000123,1,1,0,1
1000124,0,1,1,0
1000125,0,1,1,0
1000126,0,1,0,1
1000127,0,1,0,1
1000128,1,0,0,0
1000129,1,1,1,0
1000130,0,1,0,0
1000131,1,0,0,1


## Stap 2: Frequente patronen identificeren
Met een correcte datastructuur kunnen we op zoek naar de itemsets die vaak samen voorkomen.
Eerst moeten we de relatieve frequenties van alle itemcombinaties berekenen.

In **AR Mining** wordt de relatieve frequentie de **support** genoemd.

### Support

We kunnen de support van 1 item berekenen maar ook van itemsets met meerdere items.

#### Definitie
De support geeft aan hoe vaak een itemset voorkomt in de volledige dataset van **$N\ $** transacties.
De support voor item **$A\ $** kan als volgt worden berekend:
$$support(A) = \frac{\#A}{N}$$

#### Met de hand berekenen
Wanneer je de support van een itemset met de hand wilt berekenen begin je met een lege 'winkelmand' en voeg je telkens één item toe.
Je voegt dan laag per laag een item toe aan de vorige itemset. Zoals hieronder is te zien.
<br>
<img src="rescources/img/Schermafbeelding 2024-03-18 102310.png">

Bovenstaande figuur wordt een **lattice** genoemd. De lattice begint met een lege set ∅ en eindigt met de volledige set van alle items.

Het aantal itemsets dat gevormd kan worden op basis van het aantal items in de winkel kan **astronomisch groot** worden. Het totaal aantal combinaties voor een winkel die $n$ producten verkoopt kan je berekenen met volgende Python code:

In [3]:
from scipy.special import comb

n = 100

sets = sum([comb(n, k) for k in range(1, n + 1)])

print('Totaal aantal combinaties voor 100 producten {:.0f}'.format(sets))

Totaal aantal combinaties voor 100 producten 1267650600228240941970748342272


We berekenen de support handmatig voor alle itemsets die we kunnen vormen met de producten uit de transacties.
Dit doen we door te tellen hoe vaak een itemset voorkomt in de transacties en dit te delen door het aantal transacties.

- De support van alle 1 item itemsets:
    - Printer wordt 4 keer verkocht:
        - $support(Printer) = \frac{4}{10} = 0.4$
    - Papier wordt 4 keer verkocht:
        - $support(Papier) = \frac{4}{10} = 0.4$
    - Cartridge wordt 7 keer verkocht:
        - $support(Cartridge) = \frac{7}{10} = 0.7$
    - Balpen wordt 4 keer verkocht:
        - $support(Balpen) = \frac{4}{10} = 0.4$
- De support van alle 2 item itemsets:
    - $support(Printer ∧ Papier) = \frac{0}{10} = 0$
    - $support(Printer ∧ Cartridge) = \frac{3}{10} = 0.3$
    - $support(Printer ∧ Balpen) = \frac{2}{10} = 0.2$
    - $support(Papier ∧ Cartridge) = \frac{3}{10} = 0.3$
    - $support(Papier ∧ Balpen) = \frac{1}{10} = 0.1$
    - $support(Cartridge ∧ Balpen) = \frac{2}{10} = 0.2$
- De support van alle 3 item itemsets:
    - $support(Printer ∧ Papier ∧ Cartridge) = \frac{0}{10} = 0$
    - $support(Printer ∧ Papier ∧ Balpen) = \frac{0}{10} = 0$
    - $support(Printer ∧ Cartridge ∧ Balpen) = \frac{1}{10} = 0.1$
    - $support(Papier ∧ Cartridge ∧ Balpen) = \frac{1}{10} = 0.1$
- De support van alle 4 item itemsets:
    - $support(Printer ∧ Papier ∧ Cartridge ∧ Balpen) = \frac{0}{10} = 0$


| Itemset                               | Support |
|---------------------------------------|---------|
| Printer                               | 0.4     |
| Papier                                | 0.4     |
| Cartridge                             | 0.7     |
| Balpen                                | 0.4     |
| Printer ∧ Papier                      | 0       |
| Printer ∧ Cartridge                   | 0.3     |
| Printer ∧ Balpen                      | 0.2     |
| Papier ∧ Cartridge                    | 0.3     |
| Papier ∧ Balpen                       | 0.1     |
| Cartridge ∧ Balpen                    | 0.2     |
| Printer ∧ Papier ∧ Cartridge          | 0       |
| Printer ∧ Papier ∧ Balpen             | 0       |
| Printer ∧ Cartridge ∧ Balpen          | 0.1     |
| Papier ∧ Cartridge ∧ Balpen           | 0.1     |
| Printer ∧ Papier ∧ Cartridge ∧ Balpen | 0       |

De support wordt steeds kleiner naarmate dat de itemset groter wordt, dit wordt **monotonciteit** genoemd.

### AR Mining Algoritme

#### Apriori Algoritme
Het Aprirori algoritme maakt gebruik van de monotonciteiteigenschap van support om het aantal berekeningen te beperken door uit te gaan van een gekozen minimum support waaraan een combinatie moet voldoen.

Stel:
- We zijn enkel geïnteresseerd in itemsets met een support van minstens 0.2.

1. Begin vanaf de 1 item itemsets en bereken de support:
    - $support(Printer) = 0.4$
    - $support(Papier)=0.4$
    - $support(Cartridge)=0.7$
    - $support(Balpen)=0.4$
2. **Schrap** de combinaties met een support kleiner dan de minimum support.
3. Herhaal voor n item itemsets
    - $support(Printer ∧ Papier) = \frac{0}{10} = 0$
    - $support(Printer ∧ Cartridge) = \frac{3}{10} = 0.3$
    - $support(Printer ∧ Balpen) = \frac{2}{10} = 0.2$
    - $support(Papier ∧ Cartridge) = \frac{3}{10} = 0.3$
    - $support(Papier ∧ Balpen) = \frac{1}{10} = 0.1$
    - $support(Cartridge ∧ Balpen) = \frac{2}{10} = 0.2$
    - ⇒
    - $support(Printer ∧ Cartridge) = \frac{3}{10} = 0.3$
    - $support(Printer ∧ Balpen) = \frac{2}{10} = 0.2$
    - $support(Papier ∧ Cartridge) = \frac{3}{10} = 0.3$
    - $support(Cartridge ∧ Balpen) = \frac{2}{10} = 0.2$
    - ⇒
    - $support(Printer ∧ Cartridge ∧ Balpen) = 0.1$
    - ⇒
    - 
    
In Python is dit mogelijk op deze manier:

In [4]:
from mlxtend.frequent_patterns import apriori

itemsets = apriori(transacties, min_support=0.2, use_colnames=True)

itemsets

C:\Users\zouff\.venvs\DS2\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.4,(Balpen)
1,0.7,(Cartridge)
2,0.4,(Papier)
3,0.4,(Printer)
4,0.2,"(Balpen, Cartridge)"
5,0.2,"(Balpen, Printer)"
6,0.3,"(Papier, Cartridge)"
7,0.3,"(Printer, Cartridge)"


#### FP-Growth Algoritme
Het probleem met het Apriori algoritme is dat het voor elke combinatie opnieuw alle transacties moet doorlopen. Dit maakt het heel inefficiënt voor grote datasets. Omdat datasets tegenwoordig vaak heel groot zijn is er een efficiënter algoritme ontwikkeld: het FP-Growth (Frequent Pattern Growth) algoritme.

1. Bepaal de supports van alle 1 item itemsets. Het algorithme doorloop een eerste keer de transacties
    * $support(Printer) = 0.4$
    * $support(Papier) = 0.4$
    * $support(Cartridge) = 0.7$
    * $support(Balpen) = 0.4$
2. **Schrap** de items met een support kleiner dan minimumsupport (0,2). In dit geval zijn er geen items die geschrapt moeten worden.
3. **Sorteer** de items van groot naar klein volgens support ⇒ Cartridge, Printer, Papier, Balpen.
4. Bouw transactie per transactie een **boomstructuur** op.
    * Maak een eerste node. Dat is de zogenaamde **Null** node.
    * Neem de eerste transactie en sorteer de items uit die transactie volgens de volgorde uit stap 3 ⇒ $Cartridge, Printer, Balpen$
    * Plaats de items in die volgorde in de boom. Als er nog geen subnode bestaat dan voeg je die toe met een **counter** op 1. Aangezien dit de eerste transactie is zijn alle subnodes nieuw en hebben ze allemaal een counter van 1.
        * Als er al wel een subnode van het item zou bestaan dan verhoog je de counter van die subnode met 1.
<img src="rescources/img/Schermafbeelding 2024-03-28 131942.png">
    * Herhaal dit voor alle transacties.


In Python kan dit bereikt worden met de volgende code:

In [5]:
from mlxtend.frequent_patterns import fpgrowth

itemsets = fpgrowth(transacties, min_support=0.2, use_colnames=True)

itemsets

C:\Users\zouff\.venvs\DS2\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.7,(Cartridge)
1,0.4,(Printer)
2,0.4,(Balpen)
3,0.4,(Papier)
4,0.3,"(Printer, Cartridge)"
5,0.2,"(Balpen, Printer)"
6,0.2,"(Balpen, Cartridge)"
7,0.3,"(Papier, Cartridge)"


## Stap 3: AR vastleggen
Met de berekende supportgegevens is het niet moeilijk om nu verschillende **associatieregels** op te stellen.

Volgende regels komen aan bod:
- **Confidence**
- **Lift**
- **Leverage**
- **Conviction**

### Confidence
De confidence geeft de zekerheid dat een klant item B koopt als hij item A koopt. Algemeen geeft de confidence de zekerheid op item B gegeven item A.

#### Definitie
$$confidence(A \Rightarrow B) = \frac{support(A ∧ B)}{support(A)}$$
Nader beschouwd is de confidence niet meer dan een voorwaardelijke kans, want:
$$confidence(A \Rightarrow B) = P(B|A) = \frac{P(A ∧ B)}{P(A)}$$
Confidence is dan ook een getal in het interval [0,1] met 1 de beste waarde

#### Voorbeeld
We kunnen makkelijk de confidence berekenen van Cartidge op Printer:
$confidence(Printer \rightarrow Cartridge)$
Je dient dit te interpreteren als: hoe zeker zijn we er van dat men een cartridge koopt als men een Printer koopt. In dit geval kan dit als volgt berekend worden:
$$confidence(Printer \rightarrow Cartridge) = \frac{support(Printer ∧ Cartridge)}{support(Printer)} = \frac{0.3}{0.4} = 0.75$$

- Als de confidence 1 zou zijn dan is er in het verleden altijd een cartridge verkocht als er een printer werd verkocht.
- Als de confidence 0 zou zijn dan is er in het verleden nooit een cartridge verkocht als er een printer werd verkocht.

Of de confidence een geldige regel is hang af van de selectiecriteria. Vaak wordt een minimum confidence vastgelegd. Als de confidence groter is dan de minimum confidence dan is de regel geldig. Een andere optie kan zijn de top 3 regels te selecteren.

In Python kan dit als volgt berekend worden:

In [6]:
from functions.functions_Y2.Ar import *

confidence(itemsets, 'Printer', 'Cartridge')

0.7499999999999999

#### Nadelen van confidence
Wanneer je twee zeer courante items hebt die iedereen altijd aanschat (bv. water en melk) dan zal hun confidence ook heel hoog zijn. hoewel ze niet per sé iets met elkaar te maken hebben. Confidence kan dit niet onderscheiden.

### Lift
Om een AR vast te leggen is het verstandig om zowel naar de confidence als naar de support te kijken. Er is gelukkig nog een betere maatstaf om het nut en de sterkte van een AR vast te leggen: de **Lift**.

#### Definitie
$$Lift(A → B)=\frac{Confidence(A∧B)}{Support(B)}$$
De lift is de verhouding van de kans dat B voorkomt dat A t.o.v. de kans dat B zelf voorkomt.

- Als de lift > 1 dan komt B heel vaak somen met A.
- De lift is een getal tussen \[0, inf] met oneindig de beste waarde.

De lift beantwoordt de vraag: *"In welke mate levert een bepaalde regel meer zekerheid op?"*

In Python:

In [9]:
lift(itemsets, 'Balpen', 'Cartridge')

0.7142857142857143